# Technical Debt with Gemini

#### Setup
Set the Gemini API key with the provided helper function.

In [12]:
#!pip install python-dotenv
import os
from dotenv import load_dotenv, find_dotenv

def get_api_key():
    
    _ = load_dotenv(find_dotenv()) # read local .env file
    return os.getenv('GOOGLE_API_KEY')

# PaLM legacy
## import google.generativeai as palm
## palm.configure(api_key=get_api_key())

# Gemini API
import os
import google.generativeai as genai
from google.api_core import client_options as client_options_lib

genai.configure(
    api_key=get_api_key(),
    transport="rest",
    client_options=client_options_lib.ClientOptions(
        api_endpoint=os.getenv("GOOGLE_API_BASE"),
    )
)

#### Pick the model that generates text

In [13]:
models = [m for m in genai.list_models() if 'generateText' in m.supported_generation_methods]
model_bison = models[0]
model_bison

Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      top_p=0.95,
      top_k=40)

In [14]:
# Set the model to connect to the Gemini API
model_flash = genai.GenerativeModel(model_name='gemini-1.5-flash')

#### Helper function to call the PaLM API

```Python
from google.api_core import retry
@retry.Retry()
def generate_text(prompt, 
                  model=model_bison, 
                  temperature=0.0):
    return palm.generate_text(prompt=prompt,
                              model=model,
                              temperature=temperature)
```

### Helper function to call the Gemini API

In [15]:
def generate_text(prompt,
                  model=model_flash,
                  temperature=0.0):
    return model_flash.generate_content(prompt,
                                  generation_config={'temperature':temperature})

### Ask an LLM to explain a complex code base

In [16]:
#@title Complex Code Block
# Note: Taken from https://github.com/lmoroney/odmlbook/blob/63c0825094b2f44efc5c4d3226425a51990e73d6/BookSource/Chapter08/ios/cats_vs_dogs/CatVsDogClassifierSample/ModelDataHandler/ModelDataHandler.swift
CODE_BLOCK = """
// Copyright 2019 The TensorFlow Authors. All Rights Reserved.
//
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
//    http://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.

import CoreImage
import TensorFlowLite
import UIKit


/// An inference from invoking the `Interpreter`.
struct Inference {
  let confidence: Float
  let label: String
}

/// Information about a model file or labels file.
typealias FileInfo = (name: String, extension: String)

/// Information about the MobileNet model.
enum MobileNet {
  static let modelInfo: FileInfo = (name: "converted_model", extension: "tflite")
}

/// This class handles all data preprocessing and makes calls to run inference on a given frame
/// by invoking the `Interpreter`. It then formats the inferences obtained and returns the top N
/// results for a successful inference.
class ModelDataHandler {

  // MARK: - Public Properties

  /// The current thread count used by the TensorFlow Lite Interpreter.
  let threadCount: Int

  let resultCount = 1

  // MARK: - Model Parameters

  let batchSize = 1
  let inputChannels = 3
  let inputWidth = 224
  let inputHeight = 224

  // MARK: - Private Properties

  /// List of labels from the given labels file.
  private var labels: [String] = ["Cat", "Dog"]

  /// TensorFlow Lite `Interpreter` object for performing inference on a given model.
  private var interpreter: Interpreter

  /// Information about the alpha component in RGBA data.
  private let alphaComponent = (baseOffset: 4, moduloRemainder: 3)

  // MARK: - Initialization

  /// A failable initializer for `ModelDataHandler`. A new instance is created if the model and
  /// labels files are successfully loaded from the app's main bundle. Default `threadCount` is 1.
  init?(modelFileInfo: FileInfo, threadCount: Int = 1) {
    let modelFilename = modelFileInfo.name

    // Construct the path to the model file.
    guard let modelPath = Bundle.main.path(
      forResource: modelFilename,
      ofType: modelFileInfo.extension
      ) else {
        print("Failed to load the model file with name: \(modelFilename).")
        return nil
    }

    // Specify the options for the `Interpreter`.
    self.threadCount = threadCount
    var options = InterpreterOptions()
    options.threadCount = threadCount
    do {
      // Create the `Interpreter`.
      interpreter = try Interpreter(modelPath: modelPath, options: options)
    } catch let error {
      print("Failed to create the interpreter with error: \(error.localizedDescription)")
      return nil
    }

  }

  // MARK: - Public Methods

  /// Performs image preprocessing, invokes the `Interpreter`, and process the inference results.
  func runModel(onFrame pixelBuffer: CVPixelBuffer) -> [Inference]? {
    let sourcePixelFormat = CVPixelBufferGetPixelFormatType(pixelBuffer)
    assert(sourcePixelFormat == kCVPixelFormatType_32ARGB ||
      sourcePixelFormat == kCVPixelFormatType_32BGRA ||
      sourcePixelFormat == kCVPixelFormatType_32RGBA)


    let imageChannels = 4
    assert(imageChannels >= inputChannels)

    // Crops the image to the biggest square in the center and scales it down to model dimensions.
    let scaledSize = CGSize(width: inputWidth, height: inputHeight)
    guard let thumbnailPixelBuffer = pixelBuffer.centerThumbnail(ofSize: scaledSize) else {
      return nil
    }

    let outputTensor: Tensor
    do {
      // Allocate memory for the model's input `Tensor`s.
      try interpreter.allocateTensors()

      // Remove the alpha component from the image buffer to get the RGB data.
      guard let rgbData = rgbDataFromBuffer(
        thumbnailPixelBuffer,
        byteCount: batchSize * inputWidth * inputHeight * inputChannels
        ) else {
          print("Failed to convert the image buffer to RGB data.")
          return nil
      }

      // Copy the RGB data to the input `Tensor`.
      try interpreter.copy(rgbData, toInputAt: 0)

      // Run inference by invoking the `Interpreter`.
      try interpreter.invoke()

      // Get the output `Tensor` to process the inference results.
      outputTensor = try interpreter.output(at: 0)
    } catch let error {
      print("Failed to invoke the interpreter with error: \(error.localizedDescription)")
      return nil
    }

    let results = [Float32](unsafeData: outputTensor.data) ?? []

    // Process the results.
    let topNInferences = getTopN(results: results)

    // Return the inference time and inference results.
    return topNInferences
  }

  // MARK: - Private Methods

  /// Returns the top N inference results sorted in descending order.
  private func getTopN(results: [Float]) -> [Inference] {
    // Create a zipped array of tuples [(labelIndex: Int, confidence: Float)].
    let zippedResults = zip(labels.indices, results)

    // Sort the zipped results by confidence value in descending order.
    let sortedResults = zippedResults.sorted { $0.1 > $1.1 }.prefix(resultCount)

    // Return the `Inference` results.
    return sortedResults.map { result in Inference(confidence: result.1, label: labels[result.0]) }
  }

  /// Loads the labels from the labels file and stores them in the `labels` property.
  private func loadLabels(fileInfo: FileInfo) {
    let filename = fileInfo.name
    let fileExtension = fileInfo.extension
    guard let fileURL = Bundle.main.url(forResource: filename, withExtension: fileExtension) else {
      fatalError("Labels file not found in bundle. Please add a labels file with name " +
        "\(filename).\(fileExtension) and try again.")
    }
    do {
      let contents = try String(contentsOf: fileURL, encoding: .utf8)
      labels = contents.components(separatedBy: .newlines)
    } catch {
      fatalError("Labels file named \(filename).\(fileExtension) cannot be read. Please add a " +
        "valid labels file and try again.")
    }
  }

  /// Returns the RGB data representation of the given image buffer with the specified `byteCount`.
  ///
  /// - Parameters
  ///   - buffer: The pixel buffer to convert to RGB data.
  ///   - byteCount: The expected byte count for the RGB data calculated using the values that the
  ///       model was trained on: `batchSize * imageWidth * imageHeight * componentsCount`.
  ///   - isModelQuantized: Whether the model is quantized (i.e. fixed point values rather than
  ///       floating point values).
  /// - Returns: The RGB data representation of the image buffer or `nil` if the buffer could not be
  ///     converted.
  private func rgbDataFromBuffer(
    _ buffer: CVPixelBuffer,
    byteCount: Int
    ) -> Data? {
    CVPixelBufferLockBaseAddress(buffer, .readOnly)
    defer { CVPixelBufferUnlockBaseAddress(buffer, .readOnly) }
    guard let mutableRawPointer = CVPixelBufferGetBaseAddress(buffer) else {
      return nil
    }
    let count = CVPixelBufferGetDataSize(buffer)
    let bufferData = Data(bytesNoCopy: mutableRawPointer, count: count, deallocator: .none)
    var rgbBytes = [Float](repeating: 0, count: byteCount)
    var index = 0
    for component in bufferData.enumerated() {
      let offset = component.offset
      let isAlphaComponent = (offset % alphaComponent.baseOffset) == alphaComponent.moduloRemainder
      guard !isAlphaComponent else { continue }
      rgbBytes[index] = Float(component.element) / 255.0
      index += 1
    }

    return rgbBytes.withUnsafeBufferPointer(Data.init)

  }
}

// MARK: - Extensions

extension Data {
  /// Creates a new buffer by copying the buffer pointer of the given array.
  ///
  /// - Warning: The given array's element type `T` must be trivial in that it can be copied bit
  ///     for bit with no indirection or reference-counting operations; otherwise, reinterpreting
  ///     data from the resulting buffer has undefined behavior.
  /// - Parameter array: An array with elements of type `T`.
  init<T>(copyingBufferOf array: [T]) {
    self = array.withUnsafeBufferPointer(Data.init)
  }
}

extension Array {
  /// Creates a new array from the bytes of the given unsafe data.
  ///
  /// - Warning: The array's `Element` type must be trivial in that it can be copied bit for bit
  ///     with no indirection or reference-counting operations; otherwise, copying the raw bytes in
  ///     the `unsafeData`'s buffer to a new array returns an unsafe copy.
  /// - Note: Returns `nil` if `unsafeData.count` is not a multiple of
  ///     `MemoryLayout<Element>.stride`.
  /// - Parameter unsafeData: The data containing the bytes to turn into an array.
  init?(unsafeData: Data) {

    guard unsafeData.count % MemoryLayout<Element>.stride == 0 else { return nil }
    #if swift(>=5.0)
    self = unsafeData.withUnsafeBytes { .init($0.bindMemory(to: Element.self)) }
    #else
    self = unsafeData.withUnsafeBytes {
      .init(UnsafeBufferPointer<Element>(
        start: $0,
        count: unsafeData.count / MemoryLayout<Element>.stride
      ))
    }
    #endif  // swift(>=5.0)
  }
}
"""

In [17]:
prompt_template = """
Can you please explain how this code works?

{question}

Use a lot of detail and make it as clear as possible.
"""

In [18]:
completion = generate_text(
    prompt = prompt_template.format(question=CODE_BLOCK)
)

# Gemini API
print(completion.text)

# PaLM legacy
## print(completion.result)

This code implements a mobile image classification model using TensorFlow Lite in a Swift iOS application. Let's break down the code section by section:

**1. Imports and Structs:**

* `CoreImage`: Provides image processing capabilities.
* `TensorFlowLite`: The core library for running TensorFlow Lite models.
* `UIKit`:  Provides UI elements (though not directly used in this specific code snippet, it's necessary for the broader application context).
* `Inference`: A struct to hold the results of an inference (confidence score and label).
* `FileInfo`: A typealias for a tuple representing model or label file information (name and extension).
* `MobileNet`: An enum holding information about the specific MobileNet model used (filename).

**2. `ModelDataHandler` Class:**

This class is the heart of the inference process. It handles:

* **Model Loading:** The `init?` method loads the TensorFlow Lite model from the app's bundle.  It checks for errors during model loading and interpreter crea

#### Try it out on your own code!
- Try inputting some code into the `CODE_BLOCK` variable.

In [19]:
CODE_BLOCK = """ 
# replace this with your own code
def foo(a):
  b = a + 1
  return 2*b
"""

prompt_template = """
Can you please explain how this code works?

{question}

Use a lot of detail and make it as clear as possible.
"""

completion = generate_text(
    prompt = prompt_template.format(question=CODE_BLOCK)
)
print(completion.text)

Let's break down the Python function `foo(a)` step-by-step:

**1. Function Definition:**

```python
def foo(a):
  # ... function body ...
```

This line defines a function named `foo`.  The `def` keyword indicates a function definition.  `foo` is the function's name, and `(a)` specifies that it takes one argument (or parameter), which we'll call `a`.  The colon (`:`) signifies the start of the function's body.

**2. Local Variable Assignment:**

```python
  b = a + 1
```

Inside the function's body, this line creates a *local variable* named `b`.  A local variable is only accessible within the function where it's defined.  It assigns the value of `a + 1` to `b`.  In other words, it takes the input value `a`, adds 1 to it, and stores the result in `b`.  For example:

* If `a` is 5, then `b` becomes 6.
* If `a` is -2, then `b` becomes -1.
* If `a` is 0, then `b` becomes 1.

**3. Return Statement:**

```python
  return 2*b
```

This line is the function's *return statement*.  It specifies

### Ask an LLM to document a complex code base

In [20]:
prompt_template = """
Please write technical documentation for this code and \n
make it easy for a non swift developer to understand:

{question}

Output the results in markdown
"""

In [21]:
completion = generate_text(
    prompt = prompt_template.format(question=CODE_BLOCK)
)
print(completion.text)

## Function: `foo(a)`

This document describes the function `foo(a)`, written in Python.  This function takes a single numerical input and performs a simple calculation.

**1. Function Signature:**

```python
def foo(a):
  # ... function body ...
```

The function is defined using the `def` keyword.  It's named `foo` and accepts one argument, `a`.  We assume `a` is a number (integer or floating-point).

**2. Function Body:**

```python
  b = a + 1
  return 2*b
```

The function body consists of two lines:

* **`b = a + 1`**: This line calculates an intermediate value `b`. It adds 1 to the input value `a` and assigns the result to the variable `b`.

* **`return 2*b`**: This line calculates the final result. It multiplies the value of `b` by 2 and returns this result as the output of the function.

**3. Function Behavior:**

The function `foo(a)` performs the following steps:

1. Takes a numerical input `a`.
2. Adds 1 to `a` to get `b`.
3. Multiplies `b` by 2.
4. Returns the final result

### Try it out on your own code!
- Notice that we've modified the prompt slightly to refer to Python instead of Swift.

In [22]:
CODE_BLOCK = """ 
# replace this with your own code
def foo(a):
  b = a + 1
  return 2*b
"""

prompt_template = """
Please write technical documentation for this code and \n
make it easy for a non Python developer to understand:

{question}

Output the results in markdown
"""

completion = generate_text(
    prompt = prompt_template.format(question=CODE_BLOCK)
)
print(completion.text)

# Function: `foo(a)` - Technical Documentation

## Purpose

The function `foo(a)` takes a single numerical input (`a`) and performs a simple calculation to produce a numerical output.

## Input

*   **`a`**:  A number (integer or floating-point). This is the input value the function will use for its calculation.

## Process

1.  **Increment:** The function first adds 1 to the input value `a`, storing the result in a new variable called `b`.  ( `b = a + 1` )
2.  **Multiplication:** It then multiplies the value of `b` by 2. (`return 2 * b`)

## Output

*   The function returns a single numerical value which is twice the value of (`a + 1`).

## Example

Let's say the input `a` is 5:

1.  `b = a + 1` becomes `b = 5 + 1`, resulting in `b = 6`.
2.  `return 2 * b` becomes `return 2 * 6`, resulting in a final output of `12`.

Therefore, `foo(5)` would return `12`.


##  Code in Python

```python
def foo(a):
  b = a + 1
  return 2*b
```

##  How to Use (Conceptual, not specific to any language)